In [51]:
import json
import pandas as pd
import numpy as np
import os
import ast


In [2]:
# data = pd.read_parquet('roi_data_similarmerged_facecorrected_tracesfixed_textpadded_asrcorrected_final_clustersadded_medicalcontentdetected_rerun_uniqueidentifieradded.parquet')

# Let's start by conversations

In [6]:
data = pd.read_parquet('roi_data_similarmerged_facecorrected_tracesfixed_textpadded_asrcorrected_final_clustersadded_medicalcontentdetected_rerun_uniqueidentifieradded_CONVERSATION.parquet')
# and let's get a small chunk
data = data[0:20]

In [18]:
def parse_conversation(text):
    # Split the text by double newlines to get the separate turns of the conversation
    turns = text.split("\n\n")

    # Initialize an empty list to hold the structured conversation
    structured_conversation = []

    # Use a loop to process each turn of the conversation
    for turn in turns:
        # Split the turn into the speaker and what they said
        speaker, speech = turn.split(": ", 1)
        
        # Check if this is the first turn by "human" and prepend or append "<image>\n"
        if speaker == "User" and len(structured_conversation) == 0:
            if random.choice([True, False]):
                speech = "<image>\n" + speech  # either add at the front or at the end randomly selected like llava
            else:
                speech = speech + "\n<image>"  
              

        # Map 'User' and 'GPT' to 'human' and 'gpt' respectively
        from_whom = "human" if speaker == "User" else "gpt"

        # Add this turn to the structured conversation
        structured_conversation.append({
            "from": from_whom,
            "value": speech
        })

    return structured_conversation




# Let's assume 'data' is your DataFrame
# data = pd.read_csv('your_data.csv')  # If it's not already loaded

json_list = []
conversation_dir = 'quilt_instruction_tuning_dataset/conversation/'

for index, row in data.iterrows():
    # Assuming you have a similar method to get your conversation text from files
    file_path = os.path.join(conversation_dir, f"{row['unique_row_id']}_conversation.txt")

    try:
        with open(file_path, 'r') as file:
            conversation_text = file.read()
    except FileNotFoundError:
        print(f"File not found: {file_path}")
        continue

    # Use the function to parse conversations
    conversations = parse_conversation(conversation_text)

    # Create the record dictionary
    record = {
        "id": row['unique_row_id'],
        "image": row['image_path'].item(0),  # Assuming 'image_path' is the column name in your DataFrame
        "conversations": conversations
    }

    json_list.append(record)
    
json_data = json.dumps(json_list, indent=4)

# Write JSON data to a file
with open('trial_conversations.json', 'w') as json_file:
    json_file.write(json_data)

# Now Complex Reasoning:


In [35]:
data = pd.read_parquet('iterative_abduction_and_complex_reasoning.parquet')
data = data[0:500]
# data = data[0:20]

In [40]:
import os
import pandas as pd
import json
import random  # Import the random module

def parse_single_interaction(text):
    # Split the text by double newlines to get the separate parts of the interaction
    parts = text.split("\n\n")

    # Initialize an empty list to hold the structured conversation
    structured_conversation = []

    # The first part is always from the "User"
    user_question = parts[0].split(": ", 1)[1]  # Get the text after "User: "

    # Randomly decide whether to add "<image>\n" at the beginning or the end
    if random.choice([True, False]):
        user_question = "<image>\n" + user_question  # Prepend
    else:
        user_question = user_question + "\n<image>"  # Append

    structured_conversation.append({
        "from": "human",
        "value": user_question
    })

    # The remaining text is the response from GPT
    gpt_response = "\n\n".join(parts[1:]).split(": ", 1)[1]  # Get the text after "GPT: ", and join any separate paragraphs

    structured_conversation.append({
        "from": "gpt",
        "value": gpt_response
    })

    return structured_conversation

# Load your DataFrame here. This is just an example, replace it with your actual DataFrame loading code.
# data = pd.read_csv('your_data.csv')

json_list = []
conversation_dir = 'quilt_instruction_tuning_dataset/complex_reasoning/'

for index, row in data.iterrows():
    # Construct the file path
    file_path = os.path.join(conversation_dir, f"{row['unique_row_id']}_complex_reasoning.txt")

    try:
        # Read the conversation text from the file
        with open(file_path, 'r') as file:
            conversation_text = file.read()
    except FileNotFoundError:
        print(f"File not found: {file_path}")
        continue

    # Parse the interaction
    conversations = parse_single_interaction(conversation_text)

    # Create the record dictionary
    record = {
        "id": row['unique_row_id'],
        "image": row['image_path'].item(0),  # Make sure 'image_path' is the column name in your DataFrame
        "conversations": conversations
    }

    json_list.append(record)

# Convert the list to JSON
json_data = json.dumps(json_list, indent=4)

# Write the JSON data to a file
with open('trial_complex_reasoning.json', 'w') as json_file:
    json_file.write(json_data)


File not found: quilt_instruction_tuning_dataset/complex_reasoning/lWTeEfSE2oY_187_complex_reasoning.txt


# Detailed Description

In [45]:
data = pd.read_parquet('roi_data_similarmerged_facecorrected_tracesfixed_textpadded_asrcorrected_final_clustersadded_medicalcontentdetected_rerun_uniqueidentifieradded_DETAILED_DESCRIPTION.parquet')  
data = data[0:200]

In [47]:
# have a pre-created questions

detailed_description_questions = ["Can you provide a concise description of the histopathology image shown?",
"How would you summarize the features of the histopathology image displayed?",
"What are the key observations in the presented histopathology image?",
"Briefly outline the characteristics of the histopathology image in view.",
"In a few words, what does the histopathology image depict?",
"What essential information emerges from the histopathology image provided?",
"Could you distill the contents of the displayed histopathology image into a brief description?",
"Offer a succinct report on the histopathological aspects evident in the image.",
"Identify and briefly discuss the components of the histopathology image.",
"How would you encapsulate the findings presented in the histopathology image?",
"What insights can be concisely derived from the histopathology image on display?"]

# Function to create a structured interaction with a randomly chosen question and the GPT's response
def create_single_interaction(gpt_response):
    # Randomly select a question from the predefined list
    user_question = random.choice(detailed_description_questions)

    # Randomly decide whether to add "<image>\n" at the beginning or the end
    if random.choice([True, False]):
        user_question = "<image>\n" + user_question  # Prepend
    else:
        user_question = user_question + "\n<image>"  # Append

    # Build the structured conversation
    structured_conversation = [
        {
            "from": "human",
            "value": user_question
        },
        {
            "from": "gpt",
            "value": gpt_response
        }
    ]

    return structured_conversation

# Load your DataFrame here. This is just an example, replace it with your actual DataFrame loading code.
# data = pd.read_csv('your_data.csv')

json_list = []
description_dir = 'quilt_instruction_tuning_dataset/detailed_description/'

for index, row in data.iterrows():
    # Construct the file path for detailed descriptions
    file_path = os.path.join(description_dir, f"{row['unique_row_id']}_detailed_description.txt")

    try:
        # Read the detailed description from the file
        with open(file_path, 'r') as file:
            description_text = file.read().strip()  # Read and strip any leading/trailing whitespace
    except FileNotFoundError:
        print(f"File not found: {file_path}")
        continue

    # Create the interaction using the description as GPT's response
    interactions = create_single_interaction(description_text)

    # Create the record dictionary
    record = {
        "id": row['unique_row_id'],
        "image": row['image_path'].item(0),  # Make sure 'image_path' is the column name in your DataFrame
        "conversations": interactions
    }

    json_list.append(record)

# Convert the list to JSON
json_data = json.dumps(json_list, indent=4)

# Write the JSON data to a file
with open('trial_detailed_description.json', 'w') as json_file:
    json_file.write(json_data)


File not found: quilt_instruction_tuning_dataset/detailed_description/lPuADeCTsqo_74482_detailed_description.txt


# Iterative Abduction

In [48]:
data = pd.read_parquet('iterative_abduction_and_complex_reasoning.parquet')
data = data[0:50]

In [62]:
import re


def format_for_qa(simplified_text, image_identifier):
    segments = simplified_text.split('\n')
    structured_qa = []
    first_user_segment = True  # Flag to identify the first User segment

    for i in range(0, len(segments), 2):
        user_segment = segments[i].replace('User: ', '').strip()
        gpt_segment = segments[i + 1].replace('GPT: ', '').strip()

        # Check if this is the first User segment and insert the image identifier accordingly
        if first_user_segment:
            if random.choice([True, False]):
                user_segment = f"{image_identifier}\n{user_segment}"
            else:
                user_segment = f"{user_segment}\n{image_identifier}"
            first_user_segment = False  # Update the flag after processing the first User segment

        structured_qa.append({
            'role': 'User',
            'content': user_segment
        })

        structured_qa.append({
            'role': 'GPT',
            'content': gpt_segment
        })

    return structured_qa





json_list = []
abduction_dir = 'quilt_instruction_tuning_dataset/iterative_abduction/'

for index, row in data.iterrows():
    # Construct the file path for each conversation
    file_path = os.path.join(abduction_dir, f"{row['unique_row_id']}_iterative_abduction.txt")

    try:
        with open(file_path, 'r') as file:
            conversation_text = file.read()
    except FileNotFoundError:
        print(f"File not found: {file_path}")
        continue

    # Simplify the conversation text
    simplified_text = simplify_conversation(conversation_text)
    
    # Format the conversation for Q/A structure
    image_identifier = "<image>"  # Or whatever identifier you use for images
    conversations = format_for_qa(simplified_text, image_identifier)

    # Create the record dictionary
    record = {
        "id": row['unique_row_id'],
        "image": row['image_path'].item(0),  # Assuming 'image_path' is the column name in your DataFrame
        "conversations": conversations
    }

    json_list.append(record)

# Convert the list to JSON
json_data = json.dumps(json_list, indent=4)

# Writing JSON data to a file
with open('trial_iterative_abductions.json', 'w') as json_file:
    json_file.write(json_data)



# Merge it all and save

In [65]:
import json
import os

# List of your JSON files
json_files = [
    'trial_complex_reasoning.json',
    'trial_conversations.json',
    'trial_detailed_description.json',
    'trial_iterative_abductions.json'
]

# This list will store all the data loaded from the JSON files
merged_data = []

# Iterate through the list of files
for json_file in json_files:
    if os.path.exists(json_file):
        with open(json_file, 'r') as file:
            # Load the data from the JSON file
            data = json.load(file)
            
            # Assuming the data is a list of records, extend the merged data
            # If the structure is different, you may need to adjust this part
            merged_data.extend(data)
    else:
        print(f"Warning: {json_file} does not exist and will be skipped.")


random.shuffle(merged_data)

output_file = 'quilt_instruct.json'
with open(output_file, 'w') as file:
    json.dump(merged_data, file, indent=4)

print(f"The data has been merged and saved to {output_file}.")


The data has been merged and saved to quilt_instruct.json.


In [66]:
len(merged_data)

768

In [67]:
import json

# Define the name of the input file
input_file = 'quilt_instruct.json'

# This set will store the IDs
unique_ids = set()

# This list will store duplicate IDs if any
duplicate_ids = []

# Read the data from the JSON file
try:
    with open(input_file, 'r') as file:
        records = json.load(file)

    # Extract and check the IDs
    for record in records:
        record_id = record.get('id')
        if record_id in unique_ids:
            duplicate_ids.append(record_id)  # This ID is a duplicate.
        else:
            unique_ids.add(record_id)  # This ID is unique, so add it to the set.

    # Check if there were any duplicates
    if duplicate_ids:
        print(f"Warning: There are duplicates for the following IDs: {duplicate_ids}")
    else:
        print("Success: All IDs are unique.")

except FileNotFoundError:
    print(f"Error: {input_file} not found.")
except json.JSONDecodeError:
    print(f"Error: {input_file} is not a valid JSON file.")
except Exception as e:
    print(f"An error occurred: {e}")
